# Forest Monitoring in Niokolokoba


In this report, we analyze the forest cover in the Niokolokoba region. Through various geospatial data techniques, we're able to predict forest cover, calculate carbon emissions from deforestation, and estimate the carbon stock in the region. We use machine learning models and satellite imagery to make our predictions.



## Methodology
The methodology involves several steps:
1. **Data Collection**: Satellite imagery is collected for the Niokolokoba region.
2. **Forest Prediction**: Machine learning models are used to predict forest cover for subsequent years.
3. **Carbon Calculations**: Using the predicted forest cover, we calculate potential carbon emissions and carbon stock.
4. **Visualization**: The data is visualized on a map, providing an interactive tool to monitor forests.


In [ ]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt

def compute_ndvi(nir_path, red_path):
    with rasterio.open(nir_path) as nir_src:
        nir_data = nir_src.read(1)
    with rasterio.open(red_path) as red_src:
        red_data = red_src.read(1)
    
    # NDVI calculation
    ndvi = (nir_data - red_data) / (nir_data + red_data + 1e-8)
    return ndvi

def threshold_segmentation(ndvi_data, threshold=0.2):
    # Binary segmentation based on NDVI threshold
    # Vegetation areas will be marked as 1 and non-vegetation as 0
    return (ndvi_data > threshold).astype(np.uint8)

# Directory containing your files
dir_path = "."

# Retrieve B3 and B4 files
b3_files = sorted([os.path.join(dir_path, f) for f in os.listdir(dir_path) if 'B3' in f])
b4_files = sorted([os.path.join(dir_path, f) for f in os.listdir(dir_path) if 'B4' in f])

for b3_path, b4_path in zip(b3_files, b4_files):
    ndvi_data = compute_ndvi(b4_path, b3_path)  # B4 is NIR and B3 is Red
    
    # Display NDVI
    plt.imshow(ndvi_data, cmap='RdYlGn', vmin=-1, vmax=1)
    plt.colorbar(label='NDVI')
    plt.title(f"NDVI for {os.path.basename(b4_path)}")
    plt.axis('off')
    plt.show()
    
    segmented_data = threshold_segmentation(ndvi_data)
    
    # Display Segmentation
    plt.imshow(segmented_data, cmap='gray')
    plt.colorbar(label='Segmentation')
    plt.title(f"Segmentation for {os.path.basename(b4_path)}")
    plt.axis('off')
    plt.show()


In [ ]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import os
import matplotlib.pyplot as plt

# Define paths
dir_path = "."
b3_files = sorted([os.path.join(dir_path, f) for f in os.listdir(dir_path) if 'B3' in f])

# Get the B3 files for 2013 (or any other year you want to test)
b3_files_year = [file for file in b3_files if "2013" in file]

# Check if there are at least two files
if len(b3_files_year) < 2:
    raise ValueError("There are not enough B3 files for the year 2013 to merge.")

# Open the two raster images
src1 = rasterio.open(b3_files_year[0])
src2 = rasterio.open(b3_files_year[1])

# Merge them directly using the rasterio.merge.merge() function
merged_data, merged_transform = merge([src1, src2])

# Display the result
fig, axs = plt.subplots(1, 3, figsize=(20, 20))

show(src1, ax=axs[0], cmap='gray', title="Image 1")
show(src2, ax=axs[1], cmap='gray', title="Image 2")
show(merged_data, ax=axs[2], transform=merged_transform, cmap='gray', title="Merged")

plt.tight_layout()
plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import zipfile

# If you're in Jupyter Notebook, use the following line:
%matplotlib inline

predicted_dir = "./predicted"

# Path to the zipped .npy file
zipped_npy_path = os.path.join(predicted_dir, 'predicted_images.zip')

with zipfile.ZipFile(zipped_npy_path, 'r') as zip_ref:
    # Assuming there's only one .npy file in the zip, extract its name
    npy_file_name = zip_ref.namelist()[0]
    
    # Load the .npy file directly from the zip
    with zip_ref.open(npy_file_name) as file:
        predicted_images = np.load(file)

# The rest of your files appear to be local, so you can continue loading them as you were before:
accuracies = np.load(os.path.join(predicted_dir, 'accuracies.npy'))
precisions = np.load(os.path.join(predicted_dir, 'precisions.npy'))
recalls = np.load(os.path.join(predicted_dir, 'recalls.npy'))
f1_scores = np.load(os.path.join(predicted_dir, 'f1_scores.npy'))

predicted_image_files = sorted([os.path.join(predicted_dir, f) for f in os.listdir(predicted_dir) if f.endswith('.tif')])
years = [os.path.basename(f)[-8:-4] for f in predicted_image_files]

# Plotting the evaluation metrics
plt.figure(figsize=(10, 6))
plt.plot(years, accuracies, '-o', label='Accuracy')
plt.plot(years, precisions, '-o', label='Precision')
plt.plot(years, recalls, '-o', label='Recall')
plt.plot(years, f1_scores, '-o', label='F1 Score')
plt.legend()
plt.title('Evaluation Metrics Over Years')
plt.xlabel('Year')
plt.ylabel('Score')
plt.grid(True)
plt.savefig("./metrics_over_years.png")
plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import zipfile

# Directory paths
predicted_dir = "./predicted"
mosaic_dir = "./mosaics"

# Define b3_files and b4_files
b3_files = sorted([os.path.join(mosaic_dir, f) for f in os.listdir(mosaic_dir) if 'B3' in f])
b4_files = sorted([os.path.join(mosaic_dir, f) for f in os.listdir(mosaic_dir) if 'B4' in f])

# Path to the zipped .npy file
zipped_npy_path = os.path.join(predicted_dir, 'predicted_images.zip')

with zipfile.ZipFile(zipped_npy_path, 'r') as zip_ref:
    # Assuming there's only one .npy file in the zip, extract its name
    npy_file_name = zip_ref.namelist()[0]
    
    # Load the .npy file directly from the zip
    with zip_ref.open(npy_file_name) as file:
        predicted_images = np.load(file)

# Function to compute NDVI
def compute_ndvi(nir_band, red_band):
    return (nir_band - red_band) / (nir_band + red_band + 1e-8)

# For each year, visualize the NDVI data alongside the predicted forest cover
for idx, (b3_file, b4_file) in enumerate(zip(b3_files, b4_files)):
    # Load the NIR and RED bands
    with rasterio.open(b4_file) as nir_src:
        nir_band = nir_src.read(1)
    with rasterio.open(b3_file) as red_src:
        red_band = red_src.read(1)
    
    # Compute NDVI
    ndvi_data = compute_ndvi(nir_band, red_band)
    
    # Plot
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    
    axs[0].imshow(ndvi_data, cmap='RdYlGn', vmin=-1, vmax=1)
    axs[0].set_title(f'NDVI {b3_file[-8:-4]}')
    axs[0].axis('off')

    axs[1].imshow(predicted_images[idx], cmap='Greens', vmin=0, vmax=1)
    axs[1].set_title(f'Predicted Forest Cover {b3_file[-8:-4]}')
    axs[1].axis('off')
    
    axs[2].imshow(nir_band, cmap='gray')
    axs[2].set_title(f'NIR Band {b3_file[-8:-4]}')
    axs[2].axis('off')
    
    plt.tight_layout()
    plt.show()


In [ ]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Function to compute NDVI
def compute_ndvi(red_band, nir_band):
    return (nir_band - red_band) / (nir_band + red_band + 1e-8)

# Load the data
ndvi_images = []
nir_images = []
predicted_images = []
years = []

for b3_file, b4_file, pred_file in zip(b3_files, b4_files, predicted_image_files):
    year = os.path.basename(b3_file)[-8:-4]
    
    with rasterio.open(b3_file) as red_src:
        red_band = red_src.read(1)
        
    with rasterio.open(b4_file) as nir_src:
        nir_band = nir_src.read(1)
    
    with rasterio.open(pred_file) as pred_src:
        pred_img = pred_src.read(1)
        
    ndvi_img = compute_ndvi(red_band, nir_band)
    
    ndvi_images.append(ndvi_img)
    nir_images.append(nir_band)
    predicted_images.append(pred_img)
    years.append(year)

# Animation function
def animate(i):
    ax[0].clear()
    ax[1].clear()
    ax[2].clear()
    
    ax[0].imshow(ndvi_images[i], cmap='RdYlGn', vmin=-1, vmax=1)
    ax[0].set_title(f"NDVI: Year {years[i]}")
    
    ax[1].imshow(predicted_images[i], cmap='Greens', vmin=0, vmax=1)
    ax[1].set_title(f"Predicted Forest Cover: Year {years[i]}")
    
    ax[2].imshow(nir_images[i], cmap='gray')
    ax[2].set_title(f"NIR Band: Year {years[i]}")
    
    for a in ax:
        a.axis('off')

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ani = animation.FuncAnimation(fig, animate, frames=len(years), repeat=True)

# Display the animation
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import os
import rasterio
from PIL import Image
import folium
from folium.plugins import MiniMap
import geopandas as gpd

# Constants
PIXEL_AREA = 30 * 30
AVERAGE_CARBON_DENSITY_MATURE = 200

# Constants for Mapbox
MAPBOX_ACCESS_TOKEN = 'pk.eyJ1IjoiZGJjb29wZXIxOSIsImEiOiJjbGlveWZyeGgwNHNzM2xucWtmeHRtdjRjIn0.eR5g-CGcSLPyW_d_x-BAKw'
MAPBOX_SATELLITE_URL = f"https://api.mapbox.com/styles/v1/mapbox/satellite-streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={MAPBOX_ACCESS_TOKEN}"
MAPBOX_HYDROLOGY_URL = f"https://api.mapbox.com/styles/v1/mapbox/outdoors-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={MAPBOX_ACCESS_TOKEN}"

# Initialize folium map
center_coords = [13.0667, -12.7167]
m = folium.Map(location=center_coords, zoom_start=10, tiles=None)

# Add MiniMap
minimap = MiniMap()
m.add_child(minimap)

# Add base layers
folium.TileLayer(MAPBOX_HYDROLOGY_URL, attr="Mapbox Hydrology", name="Hydrology").add_to(m)
folium.TileLayer(MAPBOX_SATELLITE_URL, attr="Mapbox Satellite", name="Satellite Imagery").add_to(m)
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)

# Directory for predicted images
predicted_dir = "./predicted"
predicted_image_files = sorted([os.path.join(predicted_dir, f) for f in os.listdir(predicted_dir) if f.endswith('.tif')])

previous_forest_cover = None
carbon_emission_layers = []
carbon_stock_layers = []

for f in predicted_image_files:
    with rasterio.open(f) as src:
        forest_cover = src.read(1) > 0.5

        if previous_forest_cover is not None:
            deforestation = np.logical_and(previous_forest_cover, ~forest_cover)
            carbon_emission = deforestation.astype(np.float32) * AVERAGE_CARBON_DENSITY_MATURE
            carbon_emission_layers.append(carbon_emission)
        else:
            carbon_emission_layers.append(np.zeros_like(forest_cover, dtype=np.float32))

        carbon_stock = forest_cover.astype(np.float32) * AVERAGE_CARBON_DENSITY_MATURE
        carbon_stock_layers.append(carbon_stock)

        previous_forest_cover = forest_cover

# Image bounds
image_bounds = [[13.0667 - 0.5, -12.7167 - 0.5], [13.0667 + 0.5, -12.7167 + 0.5]]

for year in range(2013, 2021):
    overlay_image_path = f"./colored_images/forest_overlay_{year}.png"
    emission_image_path = f"./emission_colored_images/carbon_emission_{year}.png"
    stock_image_path = f"./stock_colored_images/carbon_stock_{year}.png"

    img = folium.raster_layers.ImageOverlay(name=f"Predicted Forest {year}", image=overlay_image_path, bounds=image_bounds, opacity=0.6, show=False)
    img.add_child(folium.Popup(f'Predicted Forest Cover for {year}'))
    img.add_to(m)

    emission_img = folium.raster_layers.ImageOverlay(name=f"Carbon Emission {year}", image=emission_image_path, bounds=image_bounds, opacity=0.6, show=False)
    emission_img.add_to(m)

    stock_img = folium.raster_layers.ImageOverlay(name=f"Carbon Stock {year}", image=stock_image_path, bounds=image_bounds, opacity=0.6, show=False)
    stock_img.add_to(m)

# Load Niokolo-Koba Park shapefile using geopandas
shp_path = "/niokolokoshp/WDPA_WDOECM_Sep2023_Public_2580_shp_0.zip"
niokolokoba_gdf = gpd.read_file(shp_path)

# Add Niokolo-Koba Park shapefile to folium map with a distinct color
style_function_park = lambda x: {'fillColor': '#32CD32', 'color': '#32CD32', 'fillOpacity': 0.5, 'weight': 0.5}
folium.GeoJson(niokolokoba_gdf, style_function=style_function_park, name="Niokolo-Koba National Park").add_to(m)

# Enhanced legend with hover functionality
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; z-index:9999; font-size:14px; background-color: white; padding: 10px; border-radius: 5px;">
    <b>Legend:</b><br>
    <i class="fa fa-square fa-1x" style="color:green"></i> Predicted Forest<br>
    <i class="fa fa-square fa-1x" style="color:red"></i> Carbon Emission (due to deforestation)<br>
    <i class="fa fa-square fa-1x" style="color:blue"></i> Carbon Stock<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Layer Control
folium.LayerControl().add_to(m)

m


In [ ]:
import numpy as np
import os
import rasterio
from matplotlib import pyplot as plt

# Directory containing the predicted images
predicted_dir = "./predicted"
predicted_image_files = sorted([os.path.join(predicted_dir, f) for f in os.listdir(predicted_dir) if f.endswith('.tif')])

# Constants
PIXEL_AREA = 30 * 30  # Assuming each pixel represents 30m x 30m
AVERAGE_CARBON_DENSITY_MATURE = 200  # Average carbon density for mature forests (tonnes per hectare)
AVERAGE_CARBON_DENSITY_YOUNG = 50  # Average carbon density for young/regrowing forests (tonnes per hectare)

forest_cover_area = []
total_carbon_stock = []
regrowth_areas = []
deforested_areas = []

previous_forest_cover = None
for f in predicted_image_files:
    with rasterio.open(f) as src:
        forest_cover = src.read(1) > 0.5
        
        # Calculate forest cover area
        forest_area = np.sum(forest_cover) * PIXEL_AREA / (10**6)  # in sq.km
        forest_cover_area.append(forest_area)
        
        # Calculate carbon stock
        carbon_stock = forest_area * AVERAGE_CARBON_DENSITY_MATURE  # in tonnes
        total_carbon_stock.append(carbon_stock)
        
        # If there's a previous year's data, calculate regrowth and deforested areas
        if previous_forest_cover is not None:
            regrowth = np.logical_and(~previous_forest_cover, forest_cover)  # Areas that were not forest but are now
            deforestation = np.logical_and(previous_forest_cover, ~forest_cover)  # Areas that were forest but are not now
            regrowth_areas.append(np.sum(regrowth) * PIXEL_AREA / (10**6))
            deforested_areas.append(np.sum(deforestation) * PIXEL_AREA / (10**6))
        else:
            regrowth_areas.append(0)
            deforested_areas.append(0)
        
        previous_forest_cover = forest_cover

# Carbon emissions due to deforestation (assuming all carbon is released upon deforestation)
carbon_emissions = np.array(deforested_areas) * AVERAGE_CARBON_DENSITY_MATURE

# Visualization
years = list(range(2013, 2021))

# Plotting forest cover area over the years
plt.figure(figsize=(10, 5))
plt.plot(years, forest_cover_area, marker='o', color='green', label="Forest Cover Area (sq.km)")
plt.title('Forest Cover Area Over the Years')
plt.xlabel('Year')
plt.ylabel('Area (sq.km)')
plt.grid(True)
plt.legend()
plt.show()

# Plotting carbon stock over the years
plt.figure(figsize=(10, 5))
plt.plot(years, total_carbon_stock, marker='o', color='blue', label="Carbon Stock (tonnes)")
plt.title('Carbon Stock Over the Years')
plt.xlabel('Year')
plt.ylabel('Carbon Stock (tonnes)')
plt.grid(True)
plt.legend()
plt.show()

# Plotting carbon emissions over the years
plt.figure(figsize=(10, 5))
plt.bar(years, carbon_emissions, color='red')
plt.title('Carbon Emissions Due to Deforestation Over the Years')
plt.xlabel('Year')
plt.ylabel('Carbon Emissions (tonnes)')
plt.grid(axis='y')
plt.show()


# NIOKOLOKOBA FOREST ANALYSIS REPORT

## Objective
The main objective of this project was to analyze the Niokolokoba forest's dynamics over the years in terms of deforestation, reforestation, carbon emissions due to deforestation, and carbon stock. We utilized satellite imagery and machine learning predictions.

---

## 1. Data Collection
- Satellite images of the Niokolokoba forest from the years 2013 to 2020 were collected.
- Predictions of forest cover were made using machine learning models and these predictions were stored as '.tif' files for each year.

---

## 2. Preliminary Data Analysis
- The images were loaded using the `rasterio` library to understand their dimensions and data type.
- The images were visualized using the `matplotlib` library.

---

## 3. Carbon Emission Calculation
### Assumptions
- Each pixel in the satellite image corresponds to an area of \(30 \times 30\) meters.
- The average carbon density for mature forests is 200 tonnes per hectare.

### Steps
- For each year, areas of deforestation were identified by comparing the forest cover of the current year with the previous year.
- Carbon emissions due to deforestation were calculated using the formula:
\[
\text{Carbon Emission} = \text{Deforested Area} \times \text{Average Carbon Density}
\]

---

## 4. Carbon Stock Calculation
- The carbon stored in the existing forest cover for each year was computed.
- The calculation was done using the formula:
\[
\text{Carbon Stock} = \text{Forest Cover Area} \times \text{Average Carbon Density}
\]

---

## 5. Visualization on Map
- An interactive map was created using the `Folium` library to visualize the predictions and calculations.
- Map layers were added for:
  - Predicted forest cover
  - Carbon emissions due to deforestation
  - Carbon stock

- A satellite base layer from Mapbox was integrated using an access token.

---

## 6. Deforestation and Reforestation Analysis
- For each year, areas of deforestation and reforestation were identified.
- Deforestation was computed as areas that had forest cover in the previous year but not in the current year.
- Reforestation was computed as areas that did not have forest cover in the previous year but had in the current year.
- These areas were visualized on the map and as a histogram to understand the forest dynamics.

---

## 7. User Interface and Interactivity
- Layer controls were added to the map to allow the user to toggle between different years and visualization layers.
- Legends and tooltips were added to provide contextual information to the user.
- Popup descriptions were integrated to offer details on each layer when hovered over.

---

## 8. Challenges and Solutions
- One challenge faced was aligning the overlay images with the base satellite layer. This was addressed by adjusting the image bounds to match the Niokolokoba forest's coordinates.
- Another challenge was ensuring that the carbon stock layer was represented in blue. After multiple iterations, the correct representation was achieved.

---

## 9. Conclusion
The project successfully provided insights into the Niokolokoba forest's health over the years. The interactive map serves as a valuable tool for stakeholders to understand forest dynamics, carbon emissions, and stocks.

---

This report provides a detailed summary of the work done. Adjustments and additions can be made based on specific requirements or further analyses conducted.


In [ ]:
import io
import ipywidgets as widgets
from IPython.display import display, clear_output
import rasterio
import numpy as np
import tensorflow as tf

# Load the trained model
model_path = "./forest_model.h5"
model = tf.keras.models.load_model(model_path)
patch_size = 50

# Widgets
upload_b3 = widgets.FileUpload(description="Upload B3", multiple=False)
upload_b4 = widgets.FileUpload(description="Upload B4", multiple=False)
predict_button = widgets.Button(description="Predict")
output = widgets.Output()
progress = widgets.IntProgress(value=0, min=0, max=10, step=1, description='Progress:', bar_style='info', orientation='horizontal')

def predict_forest_cover(b3_content, b4_content):
    # Implement the prediction logic here
    with rasterio.MemoryFile(b3_content) as memfile:
        with memfile.open() as src:
            red_data = src.read(1)

    with rasterio.MemoryFile(b4_content) as memfile:
        with memfile.open() as src:
            nir_data = src.read(1)

    ndvi_data = (nir_data - red_data) / (nir_data + red_data + 1e-8)
    # Simulating prediction time
    import time
    time.sleep(5)
    return f"NDVI Data Shape: {ndvi_data.shape}"

def on_predict_click(b):
    with output:
        clear_output()
        if upload_b3.value and upload_b4.value:
            b3_content = list(upload_b3.value.values())[0]["content"]
            b4_content = list(upload_b4.value.values())[0]["content"]
            
            progress.value = 1
            print("Predicting...")
            result = predict_forest_cover(b3_content, b4_content)
            progress.value = 10
            
            print(result)
        else:
            print("Please upload both B3 and B4 images.")

predict_button.on_click(on_predict_click)

# Display everything
display(widgets.VBox([upload_b3, upload_b4, predict_button, progress, output]))


FileUpload(value=(), description='Upload B3')

FileUpload(value=(), description='Upload B4')

Button(description='Predict', style=ButtonStyle())

IntProgress(value=0, bar_style='info', description='Progress:', max=10)

Output()